In [1]:
import json

from pymongo import MongoClient

import numpy as np
import matplotlib.pyplot as plt
import line_profiler

from helpers import *
from imputation import *

client = MongoClient("localhost", 27017)
db = client['usgs']
%matplotlib inline

In [ ]:
def extreme_values(y, continuity, high = True, low = True):
    n = y.size // 2
    res = y.copy()
    
    z = np.sort(np.abs(y))
    dz = z[1:] - z[:-1]
    
    if low:
        mask_min = dz[:n][::-1] > continuity
        if mask_min.sum() > 0:
            i_min = n - np.argmax(dz[:n][::-1] > continuity)
            thr_min = 0.5*(z[i_min] + z[i_min - 1])
        else:
            thr_min = z[0] - 1
    else:
        thr_min = z[0] - 1
        
    if high:
        mask_max = dz[n:] > continuity
        if mask_max.sum() > 0:
            i_max = n + np.argmax(dz[n:] > continuity)
            thr_max = 0.5*(z[i_max] + z[i_max + 1])
        else:
            thr_max = z[-1] + 1   
    else:
        thr_max = z[-1] + 1  
    
    idx = np.logical_or(res < thr_min, res > thr_max)
    return idx


def detect_spikes(y, min_amp = 0.5, thr = 0.25):
    mask = np.zeros(y.size, dtype = bool)
    for i in range(1, y.size - 1):
        dL = ( y[i] - y[i-1] ) / dt * 3600
        dR = ( y[i] - y[i+1] ) / dt * 3600 
        
        absL = abs(dL)
        absR = abs(dR)
        
        d = abs(absL - absR)
        
        if ( dL * dR > 0 ) and ( absL > min_amp ) and ( absR > min_amp ) and ( d < thr * max(absL, absR) ) :
            mask[i] = True
            
    return mask


def fix_spikes(y, mask):
    z = y.copy()
    idx = np.where(mask)[0]
    z[idx] = 0.5*(y[idx - 1] + y[idx + 1])
    return z

In [ ]:
sid = json.load(open('revision_list_d.json'))['to_review']

In [ ]:
# mis-alignment
2198000, 2422500, 2318700, 15743850

In [ ]:
good = [5124480, 2378500, 11057500, 6061500, 9447800, 2361000, 1022500, 3550000, 2329500, 11152000, 1372500, 3049000, 
        12043300, 10293000, 1193500, 9124500]

In [ ]:
e_raw = np.zeros(len(sid)) - 1
e_new = np.zeros(len(sid)) - 1

In [ ]:
si = 0
t_raw, z_raw, t_true, z_true = get_data(db, sid[si])
z_raw  = feet_to_meters(z_raw)
z_true = feet_to_meters(z_true)
dt, y_raw, y_true = align_measurements(t_raw, z_raw, t_true, z_true)
print(sid[si], dt)

In [ ]:
for si in range(len(sid)):
    try:
        t_raw, z_raw, t_true, z_true = get_data(db, sid[si])
    except:
        continue
    
    z_raw  = feet_to_meters(z_raw)
    z_true = feet_to_meters(z_true)
    dt, y_raw, y_true = align_measurements(t_raw, z_raw, t_true, z_true)
    
    if dt < 300:
        continue

    z = z_raw.copy()
    mask = extreme_values(z, continuity = 0.13, high = True, low = True)
    z[mask] = -1

    z = fill_gaps(z,  max_gap = 30 * 86400 // dt, spike_size = 2, window_size = 10)
    dt, y_new, _ = align_measurements(t_raw, z, t_true, z_true)

    y_raw  = fill_gaps(y_raw,  max_gap = 30 * 86400 // dt, spike_size = 2, window_size = 10)
    y_true = fill_gaps(y_true, max_gap = 30 * 86400 // dt, spike_size = 2, window_size = 10)
    y_new  = fill_gaps(y_new,  max_gap = 30 * 86400 // dt, spike_size = 2, window_size = 10)

    is_spike = detect_spikes(y_new, min_amp = 0.5, thr = 0.25)
    y_new = fix_spikes(y_new, is_spike)

    e_raw[si] = xae(y_raw, y_true)
    e_new[si] = xae(y_new, y_true)
    
    print(sid[si], '\t%.4f  \t%.4f' % (e_raw[si], e_new[si]))

In [ ]:
diff = e_raw - e_new
mask = np.logical_not(np.logical_or(np.isnan(e_raw), np.isnan(e_new)))
sorted(list(zip(diff[diff < 0], np.array(sid)[diff < 0])))

In [ ]:
t_raw, z_raw, t_true, z_true = get_data(db, 9418500)

z_raw  = feet_to_meters(z_raw)
z_true = feet_to_meters(z_true)
dt, y_raw, y_true = align_measurements(t_raw, z_raw, t_true, z_true)

z = z_raw.copy()
mask = extreme_values(z, continuity = 0.13, high = True, low = True)
z[mask] = -1

z = fill_gaps(z,  max_gap = 30 * 86400 // dt, spike_size = 2, window_size = 10)
dt, y_new, _ = align_measurements(t_raw, z, t_true, z_true)

y_raw  = fill_gaps(y_raw,  max_gap = 30 * 86400 // dt, spike_size = 2, window_size = 10)
y_true = fill_gaps(y_true, max_gap = 30 * 86400 // dt, spike_size = 2, window_size = 10)
y_new  = fill_gaps(y_new,  max_gap = 30 * 86400 // dt, spike_size = 2, window_size = 10)

is_spike = detect_spikes(y_new, min_amp = 0.5, thr = 0.25)
y_new = fix_spikes(y_new, is_spike)

plt.figure(figsize = (16,4))
plt.plot(y_raw, 'b-')
#plt.plot(y_true, 'g-')
plt.plot(y_new, 'r.-')
#plt.xlim(106000, 108000)

In [2]:
cursor = db['cites'].find()

X = list()
Y = list()

n = 30 * 86400 // 900

for c in cursor:
    sid = c['site_no']
    try:
        t_raw, z_raw, t_true, z_true = get_data(db, sid)
    except:
        continue
        
    z_raw  = feet_to_meters(z_raw)
    z_true = feet_to_meters(z_true)
    dt, x, y = align_measurements(t_raw, z_raw, t_true, z_true)
    
    if dt != 900:
        continue

    x = fill_gaps(x, max_gap = 30 * 86400 // dt, spike_size = 2, window_size = 10)
    y = fill_gaps(y, max_gap = 30 * 86400 // dt, spike_size = 2, window_size = 10)
    
    k = n // 2
    if k > 3:
        for i in range(0, x.size - n, k):        
            X.append(x[i:i+n])
            Y.append(y[i:i+n])

In [11]:
X_train = np.array(X, dtype = np.float32)
Y_train = np.array(Y, dtype = np.float32)

X_train[np.logical_or(np.isnan(X_train), np.isinf(X_train))] = 0
Y_train[np.logical_or(np.isnan(Y_train), np.isinf(Y_train))] = 0

X_train = np.minimum(np.maximum(X_train, 0), 40)
Y_train = np.minimum(np.maximum(Y_train, 0), 40)

m = np.min(X_train, axis = 0)
X_train -= m

s = np.max(X_train, axis = 0) - np.min(X_train, axis = 0)
X_train /= s

Y_train -= m
Y_train /= s

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
#Y_train = Y_train.reshape(Y_train.shape[0], Y_train.shape[1], 1)

In [28]:
import time
import theano
import theano.tensor as T

from lasagne.layers import Conv1DLayer, MaxPool1DLayer, LSTMLayer, InputLayer, FlattenLayer
from lasagne.layers import get_output, get_all_params
from lasagne.objectives import squared_error
from lasagne.nonlinearities import rectify
from lasagne.updates import adam

In [29]:
print("Building network ...")

n = X_train.shape[1]

target_var = T.fmatrix('target')
input_var  = T.ftensor3('input')

model = InputLayer(shape = (None, 1, n), input_var = input_var)
model = Conv1DLayer(model, num_filters = 16, filter_size = 3, nonlinearity = rectify, pad = 1)
model = Conv1DLayer(model, num_filters = 1, filter_size = 3, nonlinearity = rectify, pad = 1)
#model = Conv1DLayer(model, num_filters = 1,  filter_size = 1, nonlinearity = rectify)
model = FlattenLayer(model)

output = get_output(model)
loss = squared_error(output, target_var).mean()

params  = get_all_params(model, trainable = True)
train_prediction = get_output(model, deterministic = True)


Building network ...


/home/dlevin/anaconda3/lib/python3.5/site-packages/lasagne/theano_extensions/conv.py:66: UserWarning: The `image_shape` keyword argument to `tensor.nnet.conv2d` is deprecated, it has been renamed to `input_shape`.
  border_mode=border_mode)


In [ ]:
updates = adam(loss, params, learning_rate = 0.001)
train_fn = theano.function([input_var, target_var], loss, updates = updates)
val_fn = theano.function([input_var, target_var], loss)
print("... done.")

... done.


In [ ]:
print("Epoch \t| Train loss \t| Test loss \t| Time")
print("=========================================================")

def iterate_minibatches(X, Y, batch_size = 128):    
    x = np.zeros((batch_size, X.shape[1], 1), dtype = np.float32)
    y = np.zeros((batch_size, Y.shape[1]), dtype = np.float32)
    k = 0
    for i in range(X.shape[0]):
        x[k] = X[i]
        y[k] = Y[i]
        k += 1
        if k >= batch_size:
            k = 0
            idx = np.random.permutation(batch_size)
            yield x[idx], y[idx]


nb = 256
k = int(X_train.shape[0] * 0.7)

for epoch in range(100):    
    t = time.time()
    
    loss_train = 0
    loss_test  = 0
    
    i = 0
    for x, y in iterate_minibatches(X_train[:k], Y_train[:k], nb):
        loss_train += train_fn(np.swapaxes(x, 1, 2), y)
        i += 1
    loss_train /= i
    
    i = 0
    for x, y in iterate_minibatches(X_train[k:], Y_train[k:], nb):
        loss_test += val_fn(np.swapaxes(x, 1, 2), y)
        i += 1
    loss_test /= i
    
    t = time.time() - t
    
    print("%i \t| %.7f \t| %.7f \t| %.4f \t|" % (epoch, loss_train, loss_test, t))

Epoch 	| Train loss 	| Test loss 	| Time
0 	| 0.0040236 	| 0.0036040 	| 16.8308 	|
1 	| 0.0016836 	| 0.0007224 	| 16.8945 	|
2 	| 0.0008543 	| 0.0006474 	| 17.2491 	|
3 	| 0.0007946 	| 0.0006477 	| 16.8654 	|
4 	| 0.0009644 	| 0.0006220 	| 17.0670 	|
5 	| 0.0007845 	| 0.0006164 	| 16.9078 	|
6 	| 0.0007975 	| 0.0006201 	| 16.8762 	|
7 	| 0.0007492 	| 0.0006114 	| 16.8455 	|
8 	| 0.0007471 	| 0.0005957 	| 16.8266 	|
9 	| 0.0008185 	| 0.0006079 	| 16.8147 	|
10 	| 0.0007332 	| 0.0005914 	| 16.8200 	|
11 	| 0.0007015 	| 0.0005766 	| 16.8095 	|
12 	| 0.0007047 	| 0.0005908 	| 16.8986 	|
13 	| 0.0006679 	| 0.0005689 	| 16.8085 	|

In [ ]:
network = theano.function([input_var], output)

In [ ]:
y_pred = network(np.swapaxes(X_train[:1000], 1, 2))

In [ ]:
i = np.random.randint(0, 1000)
plt.figure(figsize = (16,4))
plt.plot(X_train[i,:,0], 'b.-')
plt.plot(Y_train[i], 'r-')
plt.plot( y_pred[i], 'k-', lw = 2)

In [ ]:
n = 1024
N = 10000

X = np.zeros((N, n, 1))
Y = np.zeros((N, n))

for i in range(N):
    k = np.random.randint(1, 6)
    u = np.zeros(n)
    idx = np.sort(np.random.randint(0, n, 2*k))
    
    for j in range(0, 2*k, 2):
        u[idx[j]:idx[j+1]] = 0.5 + 3*np.random.rand()

    idx = np.random.choice(np.arange(n), np.random.randint(10*k), replace = False)
    u[idx] = 0.5 + 3*np.random.rand(idx.size)

    c = 1 + 3*np.random.rand()
    y = np.zeros(n) + c

    a = np.zeros(2)
    a[0] = 0.6*np.random.rand()
    a[1] = 0.6 - a[0]
    b = np.random.rand(2)

    for j in range(2, n):
        y[j] = c + np.dot(a, y[j-2:j]) + np.dot(b, u[j-2:j]) + 0.01*np.random.randn()

    x = y.copy()

    j = np.random.randint(0, n//2 - 1)
    k = np.random.randint(0, n//3 - 1)
    a = np.random.randn()
    b = a + np.random.randn()
    x[j:j+k] += np.linspace(a, b, k) 

    idx = np.random.choice(np.arange(n), 16, replace = False)
    x[idx] += x.mean() + np.random.randn(idx.size)

    X[i,:,0] = x
    Y[i] = y